Forcer tensorflow à utiliser le CPU

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#  <strong> A - Installation de Tensorflow et tensorflow-hub </strong>

In [1]:
#!pip3 install --upgrade tensorflow-gpu
## Install TF-Hub.
#!pip3 install tensorflow-hub

#  <strong> B - Importation des modules necessaires </strong>

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import joblib
import pandas as pd
import os

#  <strong> C - importation du modèle USE entrainé </strong>

In [3]:
def USE_model():
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
    model = hub.load(module_url)
    print (f"Le module {module_url} a été chargé avec succès . . ." )
    return model

In [4]:
if __name__ == "__main__" :
    model = USE_model()

Le module https://tfhub.dev/google/universal-sentence-encoder/4 a été chargé avec succès . . .


In [5]:
if __name__ == "__main__" :
    sentence_embeddings = model(["Brice est là","Il n'est pas encore parti"])  # Si on importe la version 4 alors on doit retirer "["outputs"].numpy() "
    print(sentence_embeddings) 

tf.Tensor(
[[-0.07539281  0.04100859  0.07033908 ...  0.00422552 -0.05473046
  -0.02233353]
 [-0.07819533 -0.06255737 -0.00334348 ...  0.02908614  0.0010275
  -0.05095781]], shape=(2, 512), dtype=float32)


# <strong> D - Importation de données </strong> 

In [6]:
save = False
if save :
    data = {
                "validation_data" : {"lemma":pd.read_csv(f"validation_data_lemma.csv") , 
                                   "stem":pd.read_csv(f"validation_data_stem.csv")
                                  },
                "test_data" : {"lemma":pd.read_csv(f"test_data_lemma.csv") , 
                             "stem":pd.read_csv(f"test_data_stem.csv")
                                },
                "train_data" : {"lemma":pd.read_csv(f"train_data_lemma.csv") ,
                              "stem":pd.read_csv(f"train_data_stem.csv")
                             },
            }

# <strong> E - Sauvegarde des données vectorisées </strong> 

In [7]:
if save :
    for data_name in data.keys():
        for proc in ["lemma","stem"] :
            df = list(data[data_name][proc].text.apply( lambda sent : model([sent]).numpy()[0]).values)
            pd.DataFrame(df).to_csv(f"USE_embed_{data_name}_{proc}.csv",index = False)